In [1]:
from amadeus import Client, ResponseError
from datetime import date
import pandas as pd
from datetime import datetime
import traceback

# Initialize Amadeus client
amadeus = Client(client_id='JSLT2NK9MvK6L9gjhXxhv8oQbfSvGAy9', client_secret='hYE8hPobkZmyIFX5')


'''
List of origin and destination airport codes

Rio (SDU) => Brasilia (BSB)
Rio (SDU) => Congonhas (CGH)
Rio (SDU) => Guarulhos (GRU)
Congonhas (CGH) => Brasilia (BSB)
Guarulhos (GRU) => Brasilia (BSB)
Salvador (SAL) => Recife (REC)
Salvador (SAL) => Brasilia (BSB)
Recife (REC) => Brasilia (BSB)
Recife (REC) => Curitiba (CWB)
Curitiba (CWB) => Salvador (SAL)
Curitiba (CWB) => Brasilia (BSB)


'''


origin_codes = ['SDU', 'CGH', 'GRU','SAL','REC','CWB']  # Add more origin codes as needed
destination_codes = ['BSB', 'CGH', 'GRU','REC','SAL','CWB']  # Add more destination codes as needed

# List of specific pairs of origin and destination
specific_pairs = [('SDU', 'BSB'), ('SDU', 'CGH'), ('SDU', 'GRU'),('CGH', 'BSB'), ('GRU', 'BSB'), ('SAL', 'REC'),('SAL', 'BSB'), ('REC', 'BSB'), ('REC', 'CWB'),('CWB', 'SAL'), ('CWB', 'BSB')]

# Create an empty DataFrame to store the results
flight_data = pd.DataFrame(columns=['FlightNumber', 'FlightDate', 'Origin', 'Destination', 'AirlineCode', 'Price', 'OrderID', 'Total Seats', 'Available Seats', 'Total Revenue'])

In [2]:
#flight_datas = pd.read_csv('data_2023-08-09_18-00-10.csv')
#dfs = pd.read_csv('test.csv')

In [3]:
traveler = {
    'id': '1',
    'dateOfBirth': '1982-01-16',
    'name': {
        'firstName': 'JORGE',
        'lastName': 'GONZALES'
    },
    'gender': 'MALE',
    'contact': {
        'emailAddress': 'jorge.gonzales833@telefonica.es',
        'phones': [{
            'deviceType': 'MOBILE',
            'countryCallingCode': '34',
            'number': '480080076'
        }]
    },
    'documents': [{
        'documentType': 'PASSPORT',
        'birthPlace': 'Madrid',
        'issuanceLocation': 'Madrid',
        'issuanceDate': '2015-04-14',
        'number': '00000000',
        'expiryDate': '2030-04-14',
        'issuanceCountry': 'ES',
        'validityCountry': 'ES',
        'nationality': 'ES',
        'holder': True
    }]
}

In [4]:
# Create an empty DataFrame
df = pd.DataFrame(columns=['Origin', 'Destination', 'Carrier Code', 'Flight Number', 'Total Bookable Seats'])

# Set the search parameters and loop over specific pairs
for origin, destination in specific_pairs:
    body = {
        "originDestinations": [
            {
                "id": "1",
                "originLocationCode": origin,
                "destinationLocationCode": destination,
                "departureDateTime": {
                    "date": date.today().isoformat()
                }
            }
        ],
        "travelers": [
            {
                "id": "1",
                "travelerType": "ADULT"
            }
        ],
        "sources": [
            "GDS"
        ]
    }
    try:
        response = amadeus.shopping.availability.flight_availabilities.post(body)

        for entry in response.data:
            segments = entry['segments']
            for segment in segments:
                total_bookable_seats = 0
                for availability_class in segment['availabilityClasses']:
                    total_bookable_seats += availability_class['numberOfBookableSeats']

                segment_origin = segment['departure']['iataCode']
                segment_destination = segment['arrival']['iataCode']
                carrier_code = segment['carrierCode']
                flight_number = segment['number']
                
                if (segment_origin, segment_destination) == (origin, destination):
                    # Append data to the DataFrame
                    df = df.append({'Origin': segment_origin, 'Destination': segment_destination, 'Carrier Code': carrier_code, 'Flight Number': flight_number, 'Total Bookable Seats': total_bookable_seats}, ignore_index=True)
    except ResponseError as error:
        raise error

NetworkError: [---]

In [ ]:
df

,Origin,Destination,Carrier Code,Flight Number,Total Bookable Seats
0,SDU,BSB,LA,3789,89
1,SDU,BSB,G3,2064,64
2,SDU,BSB,G3,1191,63
3,SDU,BSB,LA,3785,82
4,SDU,BSB,CM,3513,36
...,...,...,...,...,...
66,GRU,BSB,LA,3529,82
67,GRU,BSB,G3,1418,70
68,GRU,BSB,LA,3257,112
69,GRU,BSB,LA,3259,112


In [ ]:
# Set the search parameters and loop over specific pairs
for origin, destination in specific_pairs:
    # Set the search parameters for each specific pair
    search_params = {
        'originLocationCode': origin,
        'destinationLocationCode': destination,
        'departureDate': date.today().isoformat(),
        'adults': 1,
        'currencyCode': 'USD',
        'nonStop': 'true'  # Include only direct flights without stops
    }

    try:
        # Make the API request to retrieve flight offers for the current specific pair
        flight_search = amadeus.shopping.flight_offers_search.get(**search_params)

        # Process the response and append to the DataFrame
        for offer in flight_search.data:
            # Get the first segment from the first itinerary
            segment = offer['itineraries'][0]['segments'][0]

            # Extract relevant information
            flight_number = segment['carrierCode'] + segment['number']
            flight_date = segment['departure']['at']
            airline_code = segment['carrierCode']

            # Flight Create Orders to book the flight
            booked_flight = amadeus.booking.flight_orders.post(offer, traveler).data

            booking_id = booked_flight['id']

            # Seat map to calculate sold seats
            seats = amadeus.shopping.seatmaps.get(flightOrderId={booking_id}).data

            # Initialize counters
            total_seats = 0
            available_seats = 0

            # Loop through the response data to count seats and extract carrier code and flight number
            for seatmap in seats:
                for deck in seatmap['decks']:
                    for seat in deck['seats']:
                        total_seats += 1
                        if 'seatAvailabilityStatus' in seat['travelerPricing'][0] and seat['travelerPricing'][0]['seatAvailabilityStatus'] == 'AVAILABLE':
                            available_seats += 1

            sold_seats = total_seats - available_seats

            sold_seats = int(sold_seats)
            price_total = float(offer['price']['total'])

            Revenue = sold_seats * price_total

            flight_data = flight_data.append({
                'FlightNumber': flight_number,
                'FlightDate': flight_date,
                'Origin': origin,
                'Destination': destination,
                'AirlineCode': airline_code,
                'Price': price_total,
                'OrderID': booking_id,
                'Total Seats': total_seats,
                'Available Seats': available_seats,
                'Total Revenue': Revenue
            }, ignore_index=True)

        flight_data = flight_data.drop_duplicates()

    except ResponseError as error:
            print(f"An error occurred while processing the specific pair: {origin} -> {destination}")
            print(f"Error message: {error}")
            print("Traceback:")
            traceback.print_exc()

# Get the current date and time
now = datetime.now()

# Format the current date and time as YYYY-MM-DD_HH-MM-SS
current_date_time = now.strftime('%Y-%m-%d_%H-%M-%S')

# Set the file name using the current date and time
file_name = f"data_{current_date_time}.csv"



# Save the DataFrame to the CSV file
#flight_data.to_csv(file_name, index=False)

print(f"DataFrame saved to {file_name}")

# Display the DataFrame with all the results
print(flight_data)

An error occurred while processing the specific pair: SDU -> BSB
Error message: [400]

Traceback:


Traceback (most recent call last):
  File "C:\Users\Omar\AppData\Local\Temp\ipykernel_7164\2060444027.py", line 33, in <module>
    seats = amadeus.shopping.seatmaps.get(flightOrderId={booking_id}).data
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\shopping\_seatmaps.py", line 22, in get
    return self.client.get('/v1/shopping/seatmaps', **params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 40, in get
    return self.request('GET', path, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 110, in request
    return self._unauthenticated_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Py

An error occurred while processing the specific pair: SDU -> CGH
Error message: [500]

Traceback:


Traceback (most recent call last):
  File "C:\Users\Omar\AppData\Local\Temp\ipykernel_7164\2060444027.py", line 28, in <module>
    booked_flight = amadeus.booking.flight_orders.post(offer, traveler).data
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\booking\_flight_orders.py", line 28, in post
    return self.client.post('/v1/booking/flight-orders', body)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 63, in post
    return self.request('POST', path, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 110, in request
    return self._unauthenticated_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Prog

An error occurred while processing the specific pair: SDU -> GRU
Error message: [400]

Traceback:


...

Traceback (most recent call last):
  File "C:\Users\Omar\AppData\Local\Temp\ipykernel_7164\2060444027.py", line 33, in <module>
    seats = amadeus.shopping.seatmaps.get(flightOrderId={booking_id}).data
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\shopping\_seatmaps.py", line 22, in get
    return self.client.get('/v1/shopping/seatmaps', **params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 40, in get
    return self.request('GET', path, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 110, in request
    return self._unauthenticated_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Py

An error occurred while processing the specific pair: GRU -> BSB
Error message: [500]

Traceback:


Traceback (most recent call last):
  File "C:\Users\Omar\AppData\Local\Temp\ipykernel_7164\2060444027.py", line 28, in <module>
    booked_flight = amadeus.booking.flight_orders.post(offer, traveler).data
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\booking\_flight_orders.py", line 28, in post
    return self.client.post('/v1/booking/flight-orders', body)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 63, in post
    return self.request('POST', path, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 110, in request
    return self._unauthenticated_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Prog

An error occurred while processing the specific pair: CWB -> BSB
Error message: [429]

Traceback:
DataFrame saved to data_2023-09-16_13-58-39.csv
  FlightNumber           FlightDate Origin Destination AirlineCode  Price  \
0       AD4612  2023-09-16T22:30:00    REC         BSB          AD  582.6   

                      OrderID Total Seats Available Seats  Total Revenue  
0  eJzTd9f3cQsKdfEDAAt0AnE%3D         174             160         8156.4  


Traceback (most recent call last):
  File "C:\Users\Omar\AppData\Local\Temp\ipykernel_7164\2060444027.py", line 15, in <module>
    flight_search = amadeus.shopping.flight_offers_search.get(**search_params)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\shopping\_flight_offers_search.py", line 32, in get
    return self.client.get('/v2/shopping/flight-offers', **params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 40, in get
    return self.request('GET', path, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\amadeus\mixins\http.py", line 110, in request
    return self._unauthenticated_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Om

In [ ]:
# Extract numeric part of 'FlightNumber' in flights_data
flight_data['Flight Number'] = flight_data['FlightNumber'].str.extract('(\d+)').astype(int)

df['Flight Number'] = df['Flight Number'].astype(int)

# Merge dataframes based on common columns
merged_data = flight_data.merge(df, how='left', left_on=['Origin', 'Destination', 'AirlineCode', 'Flight Number'], right_on=['Origin', 'Destination', 'Carrier Code', 'Flight Number'])

# Clear 'Available Seats' column
merged_data['Available Seats'] = 0

# Clear 'Total Revenue' column
merged_data['Total Revenue'] = 0

# Fill 'Available Seats' with 'Total Bookable Seats' where available
merged_data.loc[merged_data['Total Bookable Seats'].notnull(), 'Available Seats'] = merged_data['Total Bookable Seats']

# Drop the additional columns
merged_data.drop(['Carrier Code', 'Flight Number', 'Total Bookable Seats'], axis=1, inplace=True)

# Calculate Revenue and remove rows with negative Revenue
Revenue_indices_to_remove = []
for index, row in merged_data.iterrows():
    sold_seats = row['Total Seats'] - row['Available Seats']
    sold_seats = int(sold_seats)
    price_total = float(row['Price'])
    Revenue = sold_seats * price_total
    merged_data.at[index, 'Total Revenue'] = Revenue
    if Revenue < 0:
        Revenue_indices_to_remove.append(index)

merged_data.drop(Revenue_indices_to_remove, inplace=True)

print(merged_data)

  FlightNumber           FlightDate Origin Destination AirlineCode  Price  \
0       AD4612  2023-09-16T22:30:00    REC         BSB          AD  582.6   

                      OrderID Total Seats Available Seats  Total Revenue  
0  eJzTd9f3cQsKdfEDAAt0AnE%3D         174             168         3495.6  


In [ ]:
#merged_data = pd.read_csv('data_2023-08-09_18-00-10.csv')

In [ ]:
merged_data

,FlightNumber,FlightDate,Origin,Destination,AirlineCode,Price,OrderID,Total Seats,Available Seats,Total Revenue
0,AD4612,2023-09-16T22:30:00,REC,BSB,AD,582.6,eJzTd9f3cQsKdfEDAAt0AnE%3D,174,168,3495.6


In [ ]:
# Splitting datetime_column into date and time columns
merged_data[['Date', 'Time']] = merged_data['FlightDate'].str.split('T', expand=True)

# Converting date and time columns to datetime objects
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data['Time'] = pd.to_datetime(merged_data['Time']).dt.time

# Dropping the original datetime_column
merged_data.drop('FlightDate', axis=1, inplace=True)


print(df)

   Origin Destination Carrier Code  Flight Number Total Bookable Seats
0     SDU         BSB           LA           3789                   89
1     SDU         BSB           G3           2064                   64
2     SDU         BSB           G3           1191                   63
3     SDU         BSB           LA           3785                   82
4     SDU         BSB           CM           3513                   36
..    ...         ...          ...            ...                  ...
66    GRU         BSB           LA           3529                   82
67    GRU         BSB           G3           1418                   70
68    GRU         BSB           LA           3257                  112
69    GRU         BSB           LA           3259                  112
70    REC         BSB           AD           4612                  168

[71 rows x 5 columns]


In [ ]:
# Remove first 2 letters from FlightNumber
merged_data['FlightNumber'] = merged_data['FlightNumber'].apply(lambda x: x[2:])

In [ ]:
# Create a dictionary to map airport codes to their corresponding names
airport_mapping = {
    'SDU': 'Rio',
    'BSB': 'Brasilia',
    'CGH': 'Congonhas',
    'GRU': 'Guarulhos',
    'SAL': 'Salvador',
    'REC': 'Recife',
    'CWB': 'Curitiba'
}

# Map origin and destination codes to names
merged_data['Origin Name'] = merged_data['Origin'].map(airport_mapping)
merged_data['Destination Name'] = merged_data['Destination'].map(airport_mapping)


In [ ]:
# Define the new column order
new_column_order = [
    'AirlineCode', 'FlightNumber', 'Origin', 'Origin Name', 'Destination', 'Destination Name',
    'Date', 'Time', 'Price', 'OrderID',
    'Total Seats', 'Available Seats', 'Total Revenue'
]

# Reorder the DataFrame columns
merged_data = merged_data[new_column_order]

In [ ]:
merged_data

,AirlineCode,FlightNumber,Origin,Origin Name,Destination,Destination Name,Date,Time,Price,OrderID,Total Seats,Available Seats,Total Revenue
0,AD,4612,REC,Recife,BSB,Brasilia,2023-09-16,22:30:00,582.6,eJzTd9f3cQsKdfEDAAt0AnE%3D,174,168,3495.6


In [ ]:
print(type(merged_data['Destination Name']))

<class 'pandas.core.series.Series'>


In [ ]:
merged_data.to_csv(file_name, index=False)

In [ ]:
#flight_data.to_csv(file_name, index=False)

#flight_data = flight_data.drop(index=flight_data.index)

In [ ]:
import pandas as pd
import os
import re
from datetime import datetime

# Path to the folder containing the CSV files
folder_path = 'C:\\Users\\Omar\\Desktop\Masters Program\\Summer 2022\\Capstone project\\capstone project code'

# Path to the existing mega consolidated CSV file
existing_mega_csv_path = 'C:\\Users\\Omar\\Desktop\\Masters Program\\Summer 2022\\Capstone project\\capstone project code\\updated_mega_consolidated_data.csv'

# Regular expression pattern to extract the date from the file name
date_pattern = r'\d{4}-\d{2}-\d{2}'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file != existing_mega_csv_path]

# Read the existing mega consolidated CSV file
existing_mega_data = pd.read_csv(existing_mega_csv_path)

# Initialize an empty DataFrame to store consolidated data
consolidated_data = pd.DataFrame()

# Read the existing mega consolidated CSV file if it exists
if os.path.exists(existing_mega_csv_path):
    existing_mega_data = pd.read_csv(existing_mega_csv_path)
    
    # Convert the 'Date' column to datetime format if it exists
    if 'Date' in existing_mega_data.columns:
        existing_mega_data['Date'] = pd.to_datetime(existing_mega_data['Date'])
        existing_dates = existing_mega_data['Date'].dt.date.tolist()
    else:
        existing_dates = []
else:
    existing_dates = []


# Iterate through CSV files
for csv_file in csv_files:
    csv_path = os.path.join(folder_path, csv_file)
    
    # Extract date from the file title using regular expression
    match = re.search(date_pattern, csv_file)
    if match:
        file_date = match.group()
        file_date = datetime.strptime(file_date, '%Y-%m-%d').date()
        
        # Check if the date already exists in the existing mega CSV
        if file_date not in existing_dates:
            # Read the CSV file
            data = pd.read_csv(csv_path)
            
            # Add data to the consolidated DataFrame
            consolidated_data = pd.concat([consolidated_data, data], ignore_index=True)

# Convert 'Date' to datetime
consolidated_data['Date'] = pd.to_datetime(consolidated_data['Date'])

# Append the new data to the existing mega CSV
new_mega_data = pd.concat([existing_mega_data, consolidated_data], ignore_index=True)

# Save the updated mega consolidated data to a CSV
new_mega_data.to_csv('updated_mega_consolidated_data.csv', index=False)


KeyError: 'Date'

In [ ]:
new_mega_data

,AirlineCode,FlightNumber,Origin,Origin Name,Destination,Destination Name,Date,Time,Price,OrderID,Total Seats,Available Seats,Total Revenue
0,G3,1757,CWB,Curitiba,BSB,Brasilia,2023-08-09,18:20:00,460.7,eJzTd9cPMfO3cDIGAArSAiw%3D,186,0,85690.2
1,LA,3783,REC,Recife,BSB,Brasilia,2023-08-09,16:15:00,654.8,eJzTd9cPMfM3c44AAArzAlA%3D,212,96,75956.8
2,LA,3799,SDU,Rio,BSB,Brasilia,2023-08-09,19:30:00,475.7,eJzTd9cPMfM3dvQBAAraAj8%3D,168,111,27114.9
3,AD,4612,REC,Recife,BSB,Brasilia,2023-08-09,17:50:00,491.8,eJzTd9cPMfML8TICAAsxAk4%3D,27,21,2950.8
4,AD,4302,REC,Recife,BSB,Brasilia,2023-08-09,22:15:00,491.8,eJzTd9cPMfP3DnQGAAs5Al4%3D,174,113,29999.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,AD,4302,REC,Recife,BSB,Brasilia,2023-09-15,22:15:00,659.6,eJzTd9f3cfSIMPUCAAsaAlI%3D,27,21,3957.6
1471,AD,4612,REC,Recife,BSB,Brasilia,2023-09-15,17:50:00,659.6,eJzTd9f3cfTwMLMEAArbAjI%3D,174,21,100918.8
1472,LA,3713,CWB,Curitiba,BSB,Brasilia,2023-09-15,11:15:00,623.5,eJzTd9f3cfQI9AoEAAs2Amc%3D,168,111,35539.5
1473,G3,1757,CWB,Curitiba,BSB,Brasilia,2023-09-15,18:20:00,638.5,eJzTd9f3cfRwCnQFAAsLAlM%3D,186,0,118761.0


In [ ]:
'''# Read the CSV file into a DataFrame
data = pd.read_csv(existing_mega_csv_path)

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Sort the DataFrame by the 'Date' column in increasing order
sorted_data = data.sort_values(by='Date')

# Write the sorted DataFrame back to the CSV file
sorted_data.to_csv(existing_mega_csv_path, index=False)'''

"# Read the CSV file into a DataFrame\ndata = pd.read_csv(existing_mega_csv_path)\n\n# Convert the 'Date' column to datetime format\ndata['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')\n\n# Sort the DataFrame by the 'Date' column in increasing order\nsorted_data = data.sort_values(by='Date')\n\n# Write the sorted DataFrame back to the CSV file\nsorted_data.to_csv(existing_mega_csv_path, index=False)"